In [1]:

import os
import cv2
import json
import numpy as np
from collections import deque
root_dir = "C:/Users/TangSong/Desktop/lane"
resize_size = 120
view_limit = 400
YMAX = 116
YMIN = 3
XMAX = 117
XMIN = 2
follow_can = 3
flag_cross = 0

二值化函数，跑了下祖传种子生长法，还限制了下视野

In [2]:
def get_basemap(image):
    def preprocess(image):
        flip_img = cv2.flip(image, 0)

        gray = cv2.cvtColor(flip_img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        binary_bgr = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
        cropped_img = binary_bgr[:view_limit, :, :]
        binary_resize = cv2.resize(cropped_img, (resize_size, resize_size), interpolation=cv2.INTER_AREA)
        return binary_resize
        
    def get_start_point(image):
        max_len = 0  
        start_point = -1  
        current_len = 0  
        current_start = -1  

        for X in range(0, 119):
            if image[0][X][0] < 128:
                if current_len == 0:
                    current_start = X
                current_len += 1
            else:
                if current_len > max_len:
                    max_len = current_len
                    start_point = current_start
                current_len = 0

        if current_len > max_len:
            max_len = current_len
            start_point = current_start

        return start_point if max_len > 0 else -1

    
    def search_img(image, x, y):
        YY = 118
        XX = 118
        q = deque()
        q.append((x, y))
        basemap = np.ones((YY+1, XX+1), dtype=np.uint8)
        index_bm = np.ones((YY+1, XX+1), dtype=np.uint8)

        index_bm[y][x] = 0
        while q:
            a_x, a_y = q.popleft()

            if image[a_y][a_x][0] < 128:
                if basemap[a_y][a_x] == 1:
                    basemap[a_y][a_x] = 0
                directions = [(0, -1), (0, 1), (-1, 0), (1, 0)] 
                for dx, dy in directions:
                    nx, ny = a_x + dx, a_y + dy

                    if 0 <= nx <= XX and 0 <= ny <= YY and index_bm[ny][nx] == 1:
                        index_bm[ny][nx] = 0 
                        q.append((nx, ny))
            else:
                basemap[a_y][a_x] = 2
                
        return basemap
    
    image = preprocess(image)
    point = get_start_point(image)
    basemap = search_img(image, point, 0)
    gray_image = np.where(basemap == 0, 0, 255).astype(np.uint8)
    gray_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
    return cv2.flip(gray_image, 0)

拿一下标准二值图，这是计算偏差的关键

In [22]:
standard_origin = cv2.imread(root_dir + "/standard/standard_origin.jpg")
standard_binary = get_basemap(standard_origin)
cv2.imwrite(root_dir + "/standard/standard_binary.jpg", standard_binary)

True

根据标准二值图求一下标准左右线与中线。先写一个函数把图外的线要补一下

In [3]:
def fix_line_array(arr):
    arr = np.array(arr, dtype=float)
    y_coords = np.arange(len(arr))

    valid_mask = arr != -1
    x_valid = arr[valid_mask]
    y_valid = y_coords[valid_mask]

    k, b = np.polyfit(y_valid, x_valid, deg=1)

    for i in range(len(arr)):
        if arr[i] == -1:
            arr[i] = int(k * i + b)
    return arr


扫线函数还是得定义的，后面要用到，顺便定义一下拿标准线函数

In [4]:
def get_lr_line(image, scan_left, scan_right):
    left = [-1] * 119
    right = [-1] * 119
    for y in range(0,119):
        for x in range(scan_left, 2, -1):
            if image[y][x]<128 and image[y][x-1]>128 and image[y][x-2]>128:
                left[y] = x
                break
        for x in range(scan_right, 116):
            if image[y][x]<128 and image[y][x+1]>128 and image[y][x+2]>128:
                right[y] = x
                break
    return left, right

def get_standard(path):
    standard = cv2.imread(path)[:, :, 0]
    standard_left, standard_right = get_lr_line(standard, 60 ,60)
    return [fix_line_array(standard_left), fix_line_array(standard_right)]

现在要生成一批赛道二值化图像用来测试了

In [34]:
img_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp'}
for filename in os.listdir(root_dir+"/origin"):
    ext = os.path.splitext(filename)[1].lower()
    if ext in img_extensions:
        img_path = os.path.join(root_dir+"/origin", filename)
        img = cv2.imread(img_path)
        cv2.imwrite(os.path.join(root_dir+"/binary", filename),get_basemap(img))

search_img: 0.023593425750732422
search_img: 0.021533489227294922
search_img: 0.025063514709472656
search_img: 0.024056196212768555
search_img: 0.022046804428100586
search_img: 0.022540807723999023
search_img: 0.023704051971435547
search_img: 0.025110721588134766
search_img: 0.02302861213684082
search_img: 0.023842811584472656
search_img: 0.025107383728027344
search_img: 0.022541522979736328
search_img: 0.023059606552124023
search_img: 0.023041486740112305
search_img: 0.024048328399658203
search_img: 0.023540973663330078
search_img: 0.023716449737548828
search_img: 0.023029327392578125
search_img: 0.023163318634033203
search_img: 0.023540496826171875
search_img: 0.021556854248046875
search_img: 0.021534204483032227
search_img: 0.021541595458984375
search_img: 0.0230257511138916
search_img: 0.02160811424255371
search_img: 0.021632671356201172
search_img: 0.02339625358581543
search_img: 0.0221099853515625
search_img: 0.02353048324584961
search_img: 0.02355027198791504
search_img: 0.02467

最麻烦也是最重要的一步，找每张图像的左右线，在此之后偏差的计算条件就都满足了

找一下向左向右扫线的起始列，即向某一方向开始能到达的赛道部分最高处

In [5]:
def get_scan_line(image):
    scan_left = 118
    scan_right = 0
    max_edge_y = 119
    for x in range(0,119):
        edge_y = -1
        for y in range(118, 1 ,-1):
            if image[y][x]<128 and image[y-1][x]>128 and image[y-2][x]>128 or (y==2 and image[y-2][x]<128):
                edge_y = y
                break
        if edge_y!=-1 and edge_y <= max_edge_y:
            max_edge_y = edge_y
            scan_left = x
    max_edge_y = 119
    for x in range(118,-1,-1):
        edge_y = -1
        for y in range(118, 1 ,-1):
            if (image[y][x]<128 and image[y-1][x]>128 and image[y-2][x]>128) or (y==2 and image[y-2][x]<128):
                edge_y = y
                break
        if edge_y!=-1 and edge_y <= max_edge_y:
            max_edge_y = edge_y
            scan_right = x
    return scan_left, scan_right

简单地对线段滤下波，然后找下主成分与长度，方便后面算偏差

In [6]:
def process_line(line, max_gap=10):
    max_len = 0
    max_start = 0
    current_start = None
    current_len = 0
    prev_val = None

    for i, val in enumerate(line):
        if val == -1:
            if current_start is not None:
                if current_len > max_len:
                    max_len = current_len
                    max_start = current_start
                current_start = None
                current_len = 0
                prev_val = None
            continue

        if current_start is None:
            current_start = i
            current_len = 1
            prev_val = val
        else:
            if abs(val - prev_val) > max_gap:
                if current_len > max_len:
                    max_len = current_len
                    max_start = current_start
                current_start = i
                current_len = 1
            else:
                current_len += 1
            prev_val = val
    if current_start is not None and current_len > max_len:
        max_len = current_len
        max_start = current_start
    filled = line.copy()
    prev_val = None
    for i in range(max_start, max_start + max_len):
        if filled[i] != -1:
            prev_val = filled[i]
        elif prev_val is not None:
            filled[i] = prev_val
            
    
    if max_len < 10:return filled, 0, 0

    return filled, max_start, max_len


这届比赛起点三叉可以定死，圆环口又有数的图标可以模型判断，所以只需要在判到圆环时简单角点拉线即可，下面是找角点函数。
由于固定行进方向，固定顺时针过圆环，所以只要进环时（左线为右上角点向左下画线）离环时候切环内偏差直到右下角点消失即可就差不多了

In [7]:
def deal_annulus(image):
    def has_black_white_black(seq, start, end):
        first_white_index = -1
        for i in range(start, end, -1):
            prev, current = seq[i + 1], seq[i]
            if prev < 128 and current >= 128 and first_white_index == -1:
                first_white_index = i 
            elif first_white_index != -1 and prev >= 128 and current < 128:
                return first_white_index
        return -1 

    def change_line(image, corner):
        left_line = [-1]*119
        k = (corner[0]-108)/(corner[1]-0)
        b = corner[0] - k * corner[1] 
        if k==0: return None
        for y in range(108, corner[0]-1, -1):
            left_line[y] = int((y - b)/k)
        for y in range(corner[0]+1, -1, -1):
            for x in range(117, corner[1], -1):
                if image[y][x+1] <128 and image[y][x] < 128 and image[y][x-1] >128:
                    left_line[y] = x
                    break
        return left_line

    num = 0
    start_y = 0
    for y in range(114,5,-1):
        if abs(int(image[y][118])-int(image[y-1][118])) > 128:
            num+=1
            start_y = y
    if image[114][118] > 128 or num!=1:return None
    num = 0
    corner_y = -1
    corner_x = -1
    for y in range(start_y, 118):
        x = has_black_white_black(image[y], 117, -1)
        if x!=-1:
            num += 1
            corner_x = x
            corner_y = y
        else:break
    if num>=3 and corner_x!=-1:return change_line(image, [corner_y, corner_x])
    else:return None

一些画线可视化的工具函数

In [8]:
# 这个函数能够展示get_scan_line的结果
def draw_scan_lines(image, scan_left, scan_right):
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    height = image.shape[0]
    cv2.line(image, (scan_left, 0), (scan_left, height - 1), (0, 255, 0), 1)
    cv2.line(image, (scan_right, 0), (scan_right, height - 1), (0, 0, 255), 1)
    return image
# 这个函数能够展示get_lr_line的结果
def draw_lr_lines(image, left, right):
    height = len(left)
    for y in range(height):
        if left[y] != -1:
            cv2.circle(image, (left[y], y), radius=0, color=(0, 255, 0), thickness=3)
        if right[y] != -1:
            cv2.circle(image, (right[y], y), radius=0, color=(0, 0, 255), thickness=3)
    return image
# 这个函数能够展示process_line与calculate_dev的结果
def draw_main_line(image, struct_lr, dev):
    left_struct = struct_lr[0]
    right_struct = struct_lr[1]
    height = len(left_struct[0])
    for y in range(height):
        x = left_struct[0][y]
        if x == -1:
            continue
        pos = (x, y)
        if left_struct[1] <= y < left_struct[1] + left_struct[2]:
            cv2.circle(image, pos, radius=1, color=(0, 255, 0), thickness=3)
    height = len(right_struct[0])
    for y in range(height):
        x = right_struct[0][y]
        if x == -1:
            continue
        pos = (x, y)
        if right_struct[1] <= y < right_struct[1] + right_struct[2]:
            cv2.circle(image, pos, radius=1, color=(0, 0, 255), thickness=3)
    cv2.putText(
        image,
        f"{int(dev)}",
        org=(40, 80),  
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.5,
        color=(255, 255, 255),
        thickness=1,
        lineType=cv2.LINE_AA
    )
    return image


读一下逆透视数组

In [9]:
def get_perspective(path, standard_lr):
    with open(path, "r") as f:
        data = json.load(f)
    arr = data["arr"]
    wid = data["wid"]
    arr2 = [0] * 119
    for y in range(0, 119):
        arr2[y] = wid / float(standard_lr[1][y] - standard_lr[0][y] + 1)
    return [arr, arr2]

现在开始要算偏差了，注意要是主成分长度决定了用什么控制线，和祖传一样，然后最小二乘法拟合

In [10]:
kk = 120
bb = 0.2
def calculate_dev(standard_lr, struct_lr, pers):
    left_struct = struct_lr[0]
    right_struct = struct_lr[1]
    standard_left = standard_lr[0]
    standard_right = standard_lr[1]
    if left_struct[2] < 10 and right_struct[2] > 10:
        return get_kb(standard_right, right_struct, pers)
    elif right_struct[2] < 10 and left_struct[2] > 10:
        return get_kb(standard_left, left_struct, pers)
    elif right_struct[2] > 10 and left_struct[2] > 10:
        return (get_kb(standard_left, left_struct, pers) + get_kb(standard_right, right_struct, pers))/2
    else:
        return 0
        
def get_kb(standard, struct, pers):
    sumx = 0
    sumy = 0
    sumxy = 0
    sumx2 = 0
    n = 0 
    x = 0
    y = 0
    for j in range(struct[1], struct[1] + struct[2]):
        n += 1
        y = (struct[0][j] - standard[j])*pers[1][j]
        x = pers[0][j]
        sumx += x
        sumy += y
        sumxy += x * y
        sumx2 += x * x
    k = (n * sumxy - sumx * sumy) / (n * sumx2 - sumx * sumx)
    b = sumy / n - k * sumx / n
    return k * kk + b * bb

除下弯度太大导致单线被识别成双线的问题

In [11]:
def delete_line(image, scan_lr, struct_lr):
    if (struct_lr[0][2]!=0 and struct_lr[1][2]!=0) and (abs(scan_lr[1]-scan_lr[0]) < 25) and (image[0][scan_lr[0]] > 128 and image[0][scan_lr[1]] > 128):
        arr_l, start_l, len_l = struct_lr[0]
        arr_r, start_r, len_r = struct_lr[1]
        x_l = arr_l[start_l] - arr_l[start_l+len_l-1]
        k_l = abs(len_l / x_l) if x_l!=0 else float('inf')
        x_r = arr_r[start_r] - arr_r[start_r+len_r-1]
        k_r = abs(len_r / x_r) if x_r!=0 else float('inf')
        if k_l-k_r>0:struct_lr[1] = arr_r, 0, 0
        else: struct_lr[0] = arr_r, 0, 0
    return struct_lr

In [ ]:
if __name__ == "__main__":
    for filename in os.listdir(root_dir+"/binary"):
        out_path = os.path.join(root_dir+"/binary", filename)
        draw_path = os.path.join(root_dir+"/draw", filename)
        
        standard_lr = get_standard(root_dir + "/standard/standard_binary.jpg")
        perspective = get_perspective(root_dir+"/standard/perspective.json", standard_lr)
        
        image = cv2.imread(out_path)
        flip_img = image[:, :, 0]
        left_scan, right_scan = get_scan_line(flip_img)
        left_line, right_line = get_lr_line(flip_img, left_scan, right_scan)
        print(len(left_line))
        
        left_temp = deal_annulus(flip_img)
        if left_temp is not None:
            left_line = left_temp
            right_line = [-1]*119
            
        struct_lr = [process_line(left_line), process_line(right_line)]
        struct_lr = delete_line(flip_img, [left_scan, right_scan], struct_lr)
        
        dev = calculate_dev(standard_lr, struct_lr, perspective)
        
        image = draw_main_line(image, struct_lr, dev)
        cv2.imwrite(draw_path, image)

[10, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 25, 26, 26, 26, 27, 27, 26, 26, 26, 26, 25, 25, 24, 24, 23, 23, 22, 22, 21, 21, 20, 20, 19, 19, 18, 17, 17, 16, 16, 15, 15, 14, 14, 13, 13, 12, 12, 11, 10, 10, 9, 9, 8, 8, 7, 7, 6, 6, 5, 5, 4, 3, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[10, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 25, 26, 26, 26, 27, 27, 26, 26, 26, 26, 25, 25, 24, 24, 23, 23, 22, 22, 21, 21, 20, 20, 19, 19, 18, 17, 17, 16, 16, 15, 15, 14, 14, 13, 13, 12, 12, 11, 10, 10, 9, 9, 8, 8, 7, 7, 6, 6, 5, 5, 4, 3, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[10, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25,

KeyboardInterrupt: 

将代码整合为循迹模块放到jetson上测试时，竟然一次算偏差就要450ms，虽然能跑完全程，但是很占用cpu，而且限制了速度提升的空间

这一切都是因为沟槽的py，我稍微优化了部分函数，思路是将大量循环转为np计算的形式，并把种子生长法那部分写成c脚本，编译后用py调用

效果非常的好，现在算偏差只需要5ms

In [ ]:
import ctypes
lib = ctypes.CDLL(root_dir+'/lib/floodfill.dll')
lib.search_img.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=3, flags='C_CONTIGUOUS'), 
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=2, flags='C_CONTIGUOUS'), 
    ctypes.c_int,
    ctypes.c_int 
]

lib.search_img.restype = None

def search_img_py(image, start_x, start_y):
    basemap = np.ones((119, 119), dtype=np.uint8)
    lib.search_img(image, basemap, start_x, start_y)
    return basemap

In [ ]:
def get_basemap(image):
    def preprocess(image):
        flip_img = cv2.flip(image, 0)

        gray = cv2.cvtColor(flip_img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        binary_bgr = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
        cropped_img = binary_bgr[:view_limit, :, :]
        binary_resize = cv2.resize(cropped_img, (resize_size, resize_size), interpolation=cv2.INTER_AREA)
        return binary_resize
        
    def get_start_point(image):
        max_len = 0  
        start_point = -1  
        current_len = 0  
        current_start = -1  

        for X in range(0, 119):
            if image[0][X][0] < 128:
                if current_len == 0:
                    current_start = X
                current_len += 1
            else:
                if current_len > max_len:
                    max_len = current_len
                    start_point = current_start
                current_len = 0

        if current_len > max_len:
            max_len = current_len
            start_point = current_start

        return start_point if max_len > 0 else -1

    
    image = preprocess(image)
    point = get_start_point(image)
    basemap = search_img_py(image, point, 0)
    gray_image = np.where(basemap == 0, 0, 255).astype(np.uint8)
    gray_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
    return cv2.flip(gray_image, 0)

In [ ]:
def get_lr_line(image, scan_left, scan_right):
    image = np.asarray(image, dtype=np.uint8)
    H = 119
    W = 119

    left = np.full(H, -1, dtype=np.int32)
    right = np.full(H, -1, dtype=np.int32)

    if scan_left >= 2:
        x_left_range = np.arange(scan_left, 1, -1)
        for i, x in enumerate(x_left_range):
            cond = (image[:, x] < 128) & (image[:, x-1] > 128) & (image[:, x-2] > 128)
            update_mask = (left == -1) & cond
            left[update_mask] = x
    if scan_right <= 116:
        x_right_range = np.arange(scan_right, 117)
        for i, x in enumerate(x_right_range):
            cond = (image[:, x] < 128) & (image[:, x+1] > 128) & (image[:, x+2] > 128)
            update_mask = (right == -1) & cond
            right[update_mask] = x

    return left.tolist(), right.tolist()

In [ ]:
def get_scan_line(image):
    image = np.asarray(image, dtype=np.uint8)
    h, w = image.shape
    col_y      = image[2:, :]
    col_y_m1   = image[1:-1, :]
    col_y_m2   = image[0:-2, :]
    condition  = (col_y < 128) & (col_y_m1 > 128) & (col_y_m2 > 128)

    first_edge = np.argmax(condition, axis=0) + 2
    has_edge = condition.any(axis=0)
    edge_y = np.where(has_edge, first_edge, h)

    edge_y = np.where((edge_y == h) & (image[0, :] < 128), 2, edge_y)
    min_edge_y = edge_y.min()
    candidates = np.where(edge_y == min_edge_y)[0]
    if len(candidates) > 0:
        scan_left = int(candidates[0])
        scan_right = int(candidates[-1])
    else:
        scan_left, scan_right = 118, 0

    return scan_left, scan_right